In [1]:
import numpy as np
from objg import objg
import math

In [2]:
objg(np.array([0, 0, 0, 0]), 4)

(array([[102., 100., 100., 100.],
        [100., 120., 100., 100.],
        [100., 100., 102., 100.],
        [100., 100., 100., 120.]]),)

In [3]:
import numpy as np
import math
import globals
def StepSize(fun, x, d, alfa, params):
    # fun a pointer to a function (such as obja, objb, objc)
    # x a dictionary that on input has x['p'] set to the starting point values x = {'p': [-1.2, 1.0]}
    # and x['f'] and x['g'] set to the function and gradient values respectively.
    # d a vector containing the search direction
    # alfa the initial value of the step length
    # params a dictionary containing parameter values for the routine
    # params = {'ftol': 0.1,'gtol': 0.7,'xtol': 1e-6,: 100};
    #     Note that ftol and gtol were referred to as c1 and c2 in lectures. Return values are a
    #     dictionary inform that contains information about the run ('iter', 'status') and a new
    #     dictionary x containing the final solution values and function and gradient at that point.
    
    alphaL = 0
    alphaR = 1000
    xf = fun(x['p'], 1)[0]
    xg = fun(x['p'], 2)[0]
    
    for itera in range(params['maxit']):
        fi_k_t = fun(x['p'] + alfa * d, 1)[0]
        
        if fi_k_t > (xf + params['ftol'] * alfa * xg.dot(d)):
            
            alphaR = alfa
            alfa = (alphaL + alphaR) / 2
            if abs(alphaL - alphaR) < params['xtol']:
                alpha_s = 1000
                status = 'ERROR'
                x_p = x['p'] + alpha_s * d
                x_n = {'p': x_p}
                #print(alphaL, alphaR, abs(alphaL - alphaR), itera)
                return x_n, alpha_s, itera, status
            else:
                continue
        else:
            fi_k_t_d = fun(x['p'] + alfa * d, 2)[0].dot(d)
            if fi_k_t_d >= params['gtol'] * xg.dot(d):
                alpha_s = alfa
                status = 'OK'
                x_p = x['p'] + alpha_s * d
                x_n = {'p': x_p}
                return x_n, alpha_s, itera, status
            else:
                alpha_L = alfa
            if alphaR > 500:
                alfa = 2 * alpha_L
                continue
    if itera == (params['maxit']-1):
        alpha_s = 1000
        status = 'ERROR'
        x_p = x['p'] + alpha_s * d
        x_n = {'p': x_p}
        return x_n, alpha_s, itera, status

In [4]:
def SteepDescent(fun, x, sdparams):
    # sdparams = {'maxit': 1000,'toler': 1.0e-4}
    # The output inform is a dictionary containing two fields, status is 1 if the gradient tolerance
    # was achieved and 0 if not, iter is the number of steps taken and x is the solution structure,
    # with point, function and gradient evaluations at the solution point).
    Status = 0
    alpha = 1
    params = {'ftol': 0.1,'gtol': 0.7,'xtol': 1e-6,'maxit': 100}
    
    for Itera in range(sdparams['maxit']):
        d = -fun(x['p'], 2)[0]
        #print('Iternation-xk-dk-alpha:\n',Itera, x, d, alpha)
        x_n, alpha_s, itera, status = StepSize(fun, x, d, alpha, params)
        if status == 'ERROR':
            Status = 0
            #print(alpha_s, x_n)
            return x_n, Itera + 1, Status
        else:
            if np.linalg.norm(x['p']-x_n['p']) <= sdparams['toler'] or np.linalg.norm(fun(x_n['p'], 2)[0]) <= 1E-5:
                Status = 1
                return x_n, Itera + 1, Status
            else:
                xg = fun(x['p'], 2)[0]
                x_ng = fun(x_n['p'], 2)[0]
                #print(alpha_s, x_n, x['g'].dot(d)*alpha/(x_n['g'].dot(-x_n['g'])))
                alpha = max(10 * params['xtol'], xg.dot(d)*alpha_s/(x_ng.dot(-x_ng)))
                x = x_n
    
    if Itera == sdparams['maxit']-1:
        Status = 0
        return x_n, Itera + 1, Status

In [5]:
import scipy
from scipy.sparse.linalg import spsolve
globals.initialize()

def Newton(fun, x, nparams):
    # sdparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'direct'}
    # The output inform is a dictionary containing two fields, status is 1 if the gradient tolerance
    # was achieved and 0 if not, iter is the number of steps taken and x is the solution structure,
    # with point, function and gradient evaluations at the solution point).
    Status = 0
    alpha = 1
    params = {'ftol': 1E-4,'gtol': 0.7,'xtol': 1e-6,'maxit': 100}
    
    for Itera in range(nparams['maxit']):
        funH = fun(x['p'], 4)[0] # Hessian
        fung = fun(x['p'], 2)[0] # Gradient
        #print(funH, fung)
        if nparams['method'] == 'direct':
            if type(funH).__name__ == 'csr_matrix':
                d = -spsolve(funH,fung)
            else:
                d = -np.linalg.solve(funH,fung)
            if d.dot(fung) >= 0:
                D = np.abs(funH.diagonal())
                D[D < 0.01] = 0.01
                D = np.diag(1./D)
                d = -D.dot(fung)
        if nparams['method'] == 'pert':
            D = np.abs(funH.diagonal())
            D[D < 0.01] = 0.01
            D = np.diag(1./D)
            d = -D.dot(fung)
            #print(D, fung, d)
            numFact = 0
            while d.dot(fung) >= 0:
                funHn = funH + math.pow(2, numFact)*np.eye(funH.shape[0])
                numFact += 1
                D = np.abs(funHn.diagonal())
                D[D < 0.01] = 0.01
                D = np.diag(1./D)
                d = -D.dot(fung)
        #print('Iternation-xk-dk-alpha:\n',Itera, x, d, alpha)
        x_n, alpha_s, itera, status = StepSize(fun, x, d, alpha, params)
        if status == 'ERROR':
            Status = 0
            return x_n, Itera + 1, Status
        else:
            if np.linalg.norm(x['p']-x_n['p']) <= nparams['toler'] or np.linalg.norm(fun(x_n['p'], 2)[0]) <= 1E-5:
                Status = 1
                return x_n, Itera + 1, Status
            else:
                xg = fun(x['p'], 2)[0]
                x_ng = fun(x_n['p'], 2)[0]
                alpha = max(10 * params['xtol'], xg.dot(d)*alpha_s/(x_ng.dot(-x_ng)))
                x = x_n
        
    if Itera == nparams['maxit']-1:
        Status = 0
        return x_n, Itera + 1, Status

In [6]:
def BFGS(fun, x, sdparams):
    # sdparams = {'maxit': 1000,'toler': 1.0e-4}
    # The output inform is a dictionary containing two fields, status is 1 if the gradient tolerance
    # was achieved and 0 if not, iter is the number of steps taken and x is the solution structure,
    # with point, function and gradient evaluations at the solution point).
    Status = 0
    alpha = 1
    params = {'ftol': 0.1,'gtol': 0.7,'xtol': 1e-6,'maxit': 100}
    B = fun(x['p'], 4)[0] # Hessian
    x_n = x
    for Itera in range(sdparams['maxit']):
        fung = fun(x['p'], 2)[0] # Gradient
        if Itera == 0:
            s = alpha * fung
            x_n['p'] = x['p'] + s
            fung_n = fun(x_n['p'], 2)[0]
            y = fung_n - fung
            D = (s.dot(y)/(y.dot(y))) * np.eye(x['p'].shape[0])
            d = -D.dot(fung)
        else:
            if type(B).__name__ == 'csr_matrix':
                d = spsolve(B,fung)
            else:
                d = -np.linalg.solve(B,fung)
        if d.dot(fung) >= 0:
            D = np.abs(fun(x['p'], 4)[0].diagonal())
            D[D < 0.01] = 0.01
            D = np.diag(1./D)
            d = -D.dot(fung)
        #print('Iternation-xk-dk-alpha:\n',Itera, x, d, alpha)
        x_n, alpha_s, itera, status = StepSize(fun, x, d, alpha, params)
        if status == 'ERROR':
            Status = 0
            return x_n, Itera + 1, Status
        else:
            s = alpha_s * d
            x_n['p'] = x['p'] + s
            if np.linalg.norm(x['p']-x_n['p']) <= sdparams['toler']:
                Status = 1
                return x_n, Itera + 1, Status
            else:
                f = fun(x_n['p'], 1)[0]
                fung_n = fun(x_n['p'], 2)[0]
                if  np.linalg.norm(fung_n, ord = np.inf)/min(1000, 1 + abs(f)) <= 1E-4:
                    Status = 1
                    return x_n, Itera + 1, Status
                else:
                    y = fung_n - fung
                    B_n = B + np.outer(y,y)/(y.dot(s)) - (B.dot(np.outer(s,s).dot(B)))/(s.dot(B.dot(s)))
                    alpha = max(10 * params['xtol'], fung.dot(d)*alpha_s/(fung_n.dot(-fung_n)))
                    B = B_n
                    x = x_n
        
    if Itera == sdparams['maxit']-1:
        Status = 0
        return x_n, Itera + 1, Status

In [7]:
import globals

globals.initialize()

def objt(x,mode):
    z = ()
    if mode & 1:
        globals.numf += 1
        z += (x[0]**2 + math.exp(x[0]),)
    if mode & 2:
        globals.numg += 1
        z += (np.array([2*x[0] + math.exp(x[0])]),)
    if mode & 4:
        globals.numH += 1
        z += (np.array([2 + math.exp(x[0])]),)
    return z

## Test Steepest Descent method

In [8]:
sdparams = {'maxit': 1000,'toler': 1.0e-4}
x_p = np.array([1.0])
x = {'p': x_p}
SteepDescent(objt,x,sdparams)

({'p': array([-0.35172651])}, 8, 1)

In [9]:
from obja import obja
sdparams = {'maxit': 1000,'toler': 1.0e-4}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
SteepDescent(obja,x,sdparams)

({'p': array([-0.49994395,  0.49998335])}, 20, 1)

In [10]:
from objb import objb
sdparams = {'maxit': 1000,'toler': 1.0e-4}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
SteepDescent(objb,x,sdparams)

({'p': array([ 0.58370683, -0.03456573])}, 175, 1)

In [11]:
from objc import objc
sdparams = {'maxit': 2000,'toler': 1.0e-4}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
SteepDescent(objc,x,sdparams)

({'p': array([0.96347374, 0.92817193])}, 1231, 1)

In [12]:
from objd import objd
sdparams = {'maxit': 3000,'toler': 1.0e-4}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
SteepDescent(objd,x,sdparams)

({'p': array([ 1.50486039, -2.24730506])}, 2298, 0)

In [13]:
SteepDescent(objg,x,sdparams)

({'p': array([-1.55072939,  2.52637133])}, 56, 1)

## Test Newton method

In [14]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'direct'}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
Newton(obja,x,nparams)

({'p': array([-0.5,  0.5])}, 1, 1)

In [15]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'pert'}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
Newton(obja,x,nparams)

({'p': array([-0.5,  0.5])}, 1, 1)

In [16]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'direct'}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
Newton(objb,x,nparams)

({'p': array([ 0.58666667, -0.03466667])}, 1, 1)

In [17]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'pert'}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
Newton(objb,x,nparams)

({'p': array([ 0.58659304, -0.0346593 ])}, 19, 1)

In [18]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'direct'}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
Newton(objc,x,nparams)

({'p': array([0.99997229, 0.99994343])}, 39, 1)

In [19]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'pert'}
x_p = np.array([-2, 1])
x = {'p': x_p}
Newton(objc,x,nparams)

({'p': array([ 0.55016334, -2.51787175])}, 67, 0)

In [20]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'direct'}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
Newton(objd,x,nparams)

({'p': array([-0.98057268, 14.75334498])}, 128, 0)

In [21]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'pert'}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
Newton(objd,x,nparams)

({'p': array([ 0.6552835 , -0.56505306])}, 83, 1)

In [22]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'direct'}
x = {'p': np.array([1,1.0])}
Newton(objg,x,nparams)

({'p': array([0.62682067, 0.37649667])}, 13, 1)

In [23]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'pert'}
x = {'p': np.array([1,1.0])}
Newton(objg,x,nparams)

({'p': array([0.62692943, 0.37637598])}, 18, 1)

In [24]:
funH = objg(x['p'], 4)[0]
np.diag(funH.diagonal())

array([[182.,   0.],
       [  0., 120.]])

In [25]:
funH = np.array([[1,1,2],[0,0.001,3],[2,1,10]])

In [26]:
funHdiag = np.abs(funH.diagonal())
funHdiag[funHdiag < 0.01] = 0.01
np.diag(1./funHdiag)

array([[  1. ,   0. ,   0. ],
       [  0. , 100. ,   0. ],
       [  0. ,   0. ,   0.1]])

## Test BFGS method

In [27]:
sdparams = {'maxit': 1000,'toler': 1.0e-4}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
BFGS(obja,x,sdparams)

({'p': array([-0.50003997,  0.5000906 ])}, 19, 1)

In [28]:
sdparams = {'maxit': 1000,'toler': 1.0e-4}
x_p = np.array([-1.2, 1])
x = {'p': x_p}
BFGS(objb,x,sdparams)

({'p': array([ 0.58660808, -0.0346141 ])}, 25, 1)

In [29]:
sdparams = {'maxit': 1000,'toler': 1.0e-4}
x_p = np.array([1.2, 1])
x = {'p': x_p}
BFGS(objc,x,sdparams)

({'p': array([1.00008273, 1.00016157])}, 598, 1)

In [30]:
sdparams = {'maxit': 1000,'toler': 1.0e-4}
x_p = np.array([2, 1])
x = {'p': x_p}
BFGS(objd,x,sdparams)

({'p': array([ -611.99355921, -1664.81041485])}, 5, 0)

In [31]:
sdparams = {'maxit': 1000,'toler': 1.0e-4}
x_p = np.array([2, 1])
x = {'p': x_p}
BFGS(objg,x,sdparams)

({'p': array([0.62702132, 0.37633042])}, 49, 1)

In [32]:
import geodesic
import geodesicDense

In [41]:
N = 5
x_p = np.random.rand(2*N)
geodesic.geodesic(x_p,4)[0]

In [34]:
np.linalg.solve(geodesic.geodesic(x_p,4)[0].toarray(),geodesic.geodesic(x_p,2)[0])

TypeError: 'geodesic' object is not subscriptable

In [35]:
N = 5
x_p = np.random.rand(2*N)
x = {'p': x_p}

In [36]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'direct'}
x_n, Itera_s, Status = Newton(geodesic.geodesic,x,nparams)
x_n, Itera_s, Status

TypeError: 'geodesic' object is not subscriptable

In [37]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'pert'}
x_n, Itera_s, Status = Newton(geodesic.geodesic,x,nparams)
x_n, Itera_s, Status

TypeError: 'geodesic' object is not subscriptable

In [38]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'direct'}
x_n, Itera_s, Status = Newton(geodesicDense.geodesic,x,nparams)
x_n, Itera_s, Status

({'p': array([-0.66971076, -0.66971105, -0.33942267, -0.33942281, -0.00913443,
         -0.0091346 ,  0.19535732,  0.19535719,  0.59758145,  0.59758138])},
 13,
 1)

In [39]:
nparams = {'maxit': 1000,'toler': 1.0e-4, 'method': 'pert'}
x_n, Itera_s, Status = Newton(geodesicDense.geodesic,x,nparams)
x_n, Itera_s, Status

({'p': array([-0.61936602, -0.62991529, -0.23875157, -0.25988886,  0.14177257,
          0.11050059,  0.42057247,  0.3991616 ,  0.71031309,  0.6996221 ])},
 80,
 1)

In [40]:
import Opt726 as Opt726
N = 100
x_p = np.random.rand(2*N)
while(np.all(np.linalg.eigvals(geodesicDense.geodesic(x_p,4)) > 0)):
    x_p = np.random.rand(2*N)
x_p

array([5.52897708e-01, 1.74859679e-01, 8.40139789e-01, 5.16996157e-01,
       5.07893072e-01, 6.44009549e-01, 1.30375255e-01, 1.70548966e-01,
       7.33044457e-01, 9.39112019e-01, 2.09283511e-01, 9.76450497e-01,
       9.23161126e-01, 9.89419681e-01, 1.47414223e-01, 2.88082591e-02,
       8.23616372e-01, 4.48501846e-01, 5.28246624e-01, 7.25691061e-01,
       2.60686557e-01, 7.60519463e-01, 3.92393049e-01, 5.58277816e-01,
       4.98707447e-01, 3.07164047e-01, 1.95149197e-01, 8.98113065e-01,
       3.51248607e-01, 2.08573539e-01, 2.93778480e-01, 7.63353620e-01,
       2.66443161e-01, 7.95664428e-01, 8.42779395e-01, 8.47953416e-01,
       7.48247610e-01, 4.07381978e-01, 2.10973426e-01, 3.11451924e-02,
       1.65299228e-01, 3.96367719e-01, 1.35390072e-01, 5.94946137e-01,
       8.92162584e-02, 9.30033369e-01, 4.70023668e-01, 3.64650436e-01,
       2.86612897e-01, 7.50946750e-04, 5.49124704e-01, 5.01233378e-01,
       4.62769040e-01, 2.87133699e-01, 9.86012464e-01, 9.71895213e-01,
      

In [ ]:
import globals
import Geodesic as Geodesic
step = 0.01
geodata = globals.geodata
Geodesic2 = Geodesic.geodesic(geodata['alpha'], geodata['beta']\
                              , (geodata['a'],geodata['b'])\
                              , (geodata['c'],geodata['d']), N)
x = np.arange(-2,2,step)
y = np.arange(-2,2,step)
X,Y = np.meshgrid(x,y)
Z = np.zeros(X.shape)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i,j] = Geodesic2.rho(X[i,j],Y[i,j])

In [ ]:
x = {'p': x_p}
sdparams = {'maxit': 2000,'toler': 1.0e-4}
x_n, Itera_s, Status = Opt726.SteepDescent(geodesic.geodesic,x,sdparams)
x_n, Itera_s, Status

In [ ]:
x_nn = x_n['p'].reshape((Geodesic2.N, 2))
x_nn = np.insert(x_nn, 0, values=(geodata['a'],geodata['b']), axis=0)
x_nn = np.insert(x_nn, x_nn.shape[0], values=(geodata['c'],geodata['d']), axis=0)

import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1,figsize=(8,8))
ax.contourf(X,Y,Z)
ax.plot(x_nn[:,0],x_nn[:,1], color = 'r', marker='o', linestyle='solid', linewidth=2, markersize=12)
ax.set_xlabel('$x$', fontsize=20)
ax.set_ylabel('$y$', fontsize=20)
ax.set_xlim([-2,2])
ax.set_ylim([-2,2])

In [ ]:
x = {'p': x_p}
nparams = {'maxit': 2000,'toler': 1.0e-4, 'method': 'direct'}
x_n, Itera_s, Status = Opt726.Newton(geodesic.geodesic,x,nparams)
x_n, Itera_s, Status

In [ ]:
x_nn = x_n['p'].reshape((Geodesic2.N, 2))
x_nn = np.insert(x_nn, 0, values=(geodata['a'],geodata['b']), axis=0)
x_nn = np.insert(x_nn, x_nn.shape[0], values=(geodata['c'],geodata['d']), axis=0)

import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1,figsize=(8,8))
ax.contourf(X,Y,Z)
ax.plot(x_nn[:,0],x_nn[:,1], color = 'r', marker='o', linestyle='solid', linewidth=2, markersize=12)
ax.set_xlabel('$x$', fontsize=20)
ax.set_ylabel('$y$', fontsize=20)
ax.set_xlim([-2,2])
ax.set_ylim([-2,2])

In [ ]:
x = {'p': x_p}
nparams = {'maxit': 2000,'toler': 1.0e-4, 'method': 'pert'}
x_n, Itera_s, Status = Opt726.Newton(geodesic.geodesic,x,nparams)
x_n, Itera_s, Status

In [ ]:
x_nn = x_n['p'].reshape((Geodesic2.N, 2))
x_nn = np.insert(x_nn, 0, values=(geodata['a'],geodata['b']), axis=0)
x_nn = np.insert(x_nn, x_nn.shape[0], values=(geodata['c'],geodata['d']), axis=0)

import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1,figsize=(8,8))
ax.contourf(X,Y,Z)
ax.plot(x_nn[:,0],x_nn[:,1], color = 'r', marker='o', linestyle='solid', linewidth=2, markersize=12)
ax.set_xlabel('$x$', fontsize=20)
ax.set_ylabel('$y$', fontsize=20)
ax.set_xlim([-2,2])
ax.set_ylim([-2,2])

In [ ]:
a = np.array([1,2,3])
b = np.array([1,2,3])
np.outer(a,b)

In [ ]:
x = {'p': x_p}
nparams = {'maxit': 2000,'toler': 1.0e-4, 'method': 'direct'}
x_n, Itera_s, Status = Opt726.BFGS(geodesic.geodesic,x,nparams)
x_n, Itera_s, Status

In [ ]:
x_nn = x_n['p'].reshape((Geodesic2.N, 2))
x_nn = np.insert(x_nn, 0, values=(geodata['a'],geodata['b']), axis=0)
x_nn = np.insert(x_nn, x_nn.shape[0], values=(geodata['c'],geodata['d']), axis=0)

import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1,figsize=(8,8))
ax.contourf(X,Y,Z)
ax.plot(x_nn[:,0],x_nn[:,1], color = 'r', marker='o', linestyle='solid', linewidth=2, markersize=12)
ax.set_xlabel('$x$', fontsize=20)
ax.set_ylabel('$y$', fontsize=20)
ax.set_xlim([-2,2])
ax.set_ylim([-2,2])

In [ ]:
import Opt726 as Opt726
N = 100
dirc = []
pert = []
for i in range(20):
    x_p = np.random.rand(2*N)
    while(np.all(np.linalg.eigvals(geodesicDense.geodesic(x_p,4)) > 0)):
        x_p = np.random.rand(2*N)
    x = {'p': x_p}
    nparams = {'maxit': 2000,'toler': 1.0e-4, 'method': 'direct'}
    x_n, Itera_s, Status = Opt726.Newton(geodesic.geodesic,x,nparams)
    dirc.append(Itera_s)
    nparams = {'maxit': 2000,'toler': 1.0e-4, 'method': 'pert'}
    x_n, Itera_s, Status = Opt726.Newton(geodesic.geodesic,x,nparams)
    pert.append(Itera_s)

In [ ]:
en = []
for i in range(20):
    en.append(i+1)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
ax.plot(en,dirc, color = 'r', marker='o', linestyle='solid', linewidth=2, markersize=12)
ax.plot(en,pert, color = 'b', marker='s', linestyle='solid', linewidth=2, markersize=12)
ax.set_xlabel('$n$', fontsize=20)
ax.set_ylabel('$iterations$', fontsize=20)
ax.set_xlim([1,20])
ax.legend(['direct','pert'])